In [6]:
import pandas as pd
import sqlite3

In [7]:
conn = sqlite3.connect('../res/tables.sql')
cursor = conn.cursor()

In [8]:
cursor.execute('''
SELECT * FROM profile
JOIN activities
ON profile.id = activities.id
''')
row = cursor.fetchall()
conn.close()

In [9]:
# convert the data from bytes to integer
dataset = map(lambda xs: [x for x in map(lambda y: int.from_bytes(y, 'little'), xs)], row)

# fetch the column names
column_names = [item[0] for item in cursor.description]

df = pd.DataFrame(data=dataset, columns=column_names)
df.head()

,id,gender,age,household size,family relationship,health,id,study time,social activities,drinker,absences
0,1,1,18,1,4,3,1,2,4,0,6
1,2,1,17,1,5,3,2,2,3,0,4
2,3,1,15,0,4,3,3,2,2,1,10
3,4,1,15,1,3,5,4,3,2,0,2
4,5,1,16,1,4,5,5,2,2,0,4


In [10]:
[item[0] for item in cursor.description]

['id',
 'gender',
 'age',
 'household size',
 'family relationship',
 'health',
 'id',
 'study time',
 'social activities',
 'drinker',
 'absences']